In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, coo_matrix, csr_array

In [2]:
## Lets make two 3D simple arrays. A dapi array with two cells, labelled 2 and 7
## and a microglia array. The microglia three processes has labels 9, 5, 6
## The two arrays have the same dimensions. Is this TRUE Christina??

dapi = [
    [2, 2, 0, 0, 7],
    [2, 2, 0, 7, 7],
    [0, 0, 0, 0, 0]
]
dapi_IFr = np.stack([dapi, dapi])
# dapi_binary = (dapi_IFr >1).astype(np.uint32)

microglia = [
    [9, 0, 5, 0, 0],
    [0, 5, 5, 5, 0],
    [0, 0, 5, 5, 6]
]
mg_filtered_IFr = np.stack([microglia, microglia])

In [3]:
assert dapi_IFr.shape == mg_filtered_IFr.shape

In [4]:
# get the number of planes. It is assumed the arrays are in ZYX format
num_planes = dapi_IFr.shape[0]

In [5]:
my_mg = []
my_nuclei = []
for d in range(num_planes):
    dapi_plane = dapi_IFr[d]
    mg_plane = mg_filtered_IFr[d]

    dapi_csr = csr_array(dapi_plane)
    mg_csr = csr_array(mg_plane)

    _inNuclei = dapi_csr[mg_csr.nonzero()]

    # get the non zero elements only
    idx = _inNuclei > 0

    _inNuclei = _inNuclei[idx]
    mg = mg_csr.data[idx]

    my_mg.append(mg.tolist())
    my_nuclei.append(_inNuclei.tolist())
    

In [6]:
# Christina, the line below is a bit cryptic, all it does is to flatten a nested list of lists
print(my_mg) # this is before the flattening
mg_labels = sum(my_mg, [])
mg_labels # this is after the flattening

[[9, 5, 5], [9, 5, 5]]


[9, 5, 5, 9, 5, 5]

In [7]:
inNuclei = sum(my_nuclei, [])
inNuclei

[2, 2, 7, 2, 2, 7]

In [8]:
# now that we have flattened everything we make a dataframe
df = pd.DataFrame({'mg_labels':mg_labels, 'inNuclei': inNuclei})
df

,mg_labels,inNuclei
0,9,2
1,5,2
2,5,7
3,9,2
4,5,2
5,5,7


In [9]:
# group now by both mg and dapi labels
df.groupby(['mg_labels','inNuclei']).size()

mg_labels  inNuclei
5          2           2
           7           2
9          2           2
dtype: int64

In [10]:
# the matrix above means that the process with label 5 overlaps with two cells, those with 
# labels 2 and 7. In total 4 pixels of the microglia process are within the boundaries 
# of those cells

### RESULT: I think this is what you want, right?

In [11]:
df.drop_duplicates().groupby('mg_labels').count()

,inNuclei
mg_labels,
5,2
9,1


In [12]:
# It shows that microglia with label 5 overlaps with 2 different nuclei 
# and microglia with label 9 with just one
# If you arelooking for an mg_label and is missing from here then there is 
# no ovelap with a nucleus